In [1]:
# uncomment code if running the script on google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import os
import re

# Checking if CUDA is available
flag_cuda = torch.cuda.is_available()

if not flag_cuda:
    print('Using CPU')
else:
    print('Using GPU')

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

Using CPU


### Set Test Files and Result Location
*Note: model is loaded from 'toTestPath' location*

In [3]:
fromTestPath = '/content/drive/MyDrive/ECE542_fa2021_Project_TerrainRecognition/TestData'
toTestPath = '/content/drive/MyDrive/Colab Notebooks/projC phase II/code/' # results folder 

In [20]:
### Helper function
def read_testFile(tf):
      test_data = pd.read_csv(tf, header = None)
      noOfCols = test_data.shape[1]
      noOfRows = test_data.shape[0]
      train_X = test_data.loc[:,:]

      start_index = 1
      end_index = 48
      image_height = 48 #46
      sample = []
      for indx in range(1,noOfRows-image_height):
        start_index = indx
        a = train_X[start_index:end_index+1].values
        r,c = a.shape
        a = a.reshape(1,r,c)
        sample.append(transform(torch.from_numpy(a).float()))
        end_index += 1
      return sample

testSubjects = []
for f in os.listdir(fromTestPath):
  if re.search('^subject_.*__x.csv$', f , flags= re.I):
    testSubjects.append(f)

# List all the test files
testFiles = [fromTestPath+'/'+x for x in testSubjects]

In [18]:
# Convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.Normalize((0.5), (0.5))
    ])

# Defining the CNN architecture
class TerrainClassifier(nn.Module):
  def __init__(self):
    super(TerrainClassifier, self).__init__()
    self.conv1 = nn.Conv2d(1,5,3,1,1)
    self.batch1 = nn.BatchNorm2d(5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(5, 4, 3,1,1)
    self.batch2 = nn.BatchNorm2d(4)
    self.fc1 = nn.Linear(4 * 48 * 6,  30)
    self.batch3 = nn.BatchNorm1d(30)
    self.fc2 = nn.Linear(30, 16)
    self.batch4 = nn.BatchNorm1d(16)
    self.fc3 = nn.Linear(16, 4)

  def forward(self, x):
    x = F.relu(self.batch1(self.conv1(x)))
    x = F.relu(self.batch2(self.conv2(x)))
    x = x.view(-1, 4*48*6) 
    x = F.relu(self.batch3(self.fc1(x)))
    x = F.relu(self.batch4(self.fc2(x)))
    x = self.fc3(x)
    return x

# Create a complete CNN
model = TerrainClassifier()

In [23]:
# Loading the best model
model.load_state_dict(torch.load(os.path.join(toTestPath,'model_cifar.pt')))

# Trigger model evaluation
model.eval()

## Prediction on test data
if len(testFiles) > 1:
  for dtFile in testFiles:
    test_data = read_testFile(dtFile)
    # print('X size: {} => {}'.format(len(test_data), test_data[0].shape))
    test_data = DataLoader(test_data, batch_size=1, shuffle=False)
    y_hat = []
    for testPackt in test_data:
      output = model(testPackt)
      _, pred = torch.max(output, 1)
      torch.no_grad()
      pred = pred.detach().numpy()
      y_hat.append(pred.tolist()[0])
  y_init = [0 for i in range(0,48)]
  y_init.extend(y_hat)
  y_hat = y_init.copy() 
  y_hat = y_hat[::4]
  pred = pd.DataFrame()
  pred = pd.DataFrame({'prediction':y_hat})
  pred.to_csv(toTestPath+'/'+dtFile.split('.')[-2].split('/')[-1].replace('x','y')+'.csv', header= None, index=False)
  print('Prediction Successful...')
elif len(testFiles) == 1:
  test_data = read_testFile(testFiles[0])
  # print('X size: {} => {}'.format(len(test_data), test_data[0].shape))
  test_data = DataLoader(test_data, batch_size=1, shuffle=False)
  y_hat = []
  for testPackt in test_data:
    output = model(testPackt)
    _, pred = torch.max(output, 1)
    torch.no_grad()
    pred = pred.detach().numpy()
    y_hat.append(pred.tolist()[0])
  y_init = [0 for i in range(0,48)]
  y_init.extend(y_hat)
  y_hat = y_init.copy() 
  y_hat = y_hat[::4]
  pred = pd.DataFrame()
  pred = pd.DataFrame({'prediction':y_hat})
  pred.to_csv(toTestPath+'/'+dtFile.split('.')[-2].split('/')[-1].replace('x','y')+'.csv', header= None, index=False)
  print('Prediction Successful...')
else:
  print('Error-->invalid test file format...')

Prediction Successful...
